In [28]:
!pip3 -qq install torch==0.4.1
!pip -qq install torchtext==0.3.1
!pip -qq install gensim==3.6.0
!pip -qq install pyldavis==2.1.2
!pip -qq install attrs==18.2.0
!wget -qq --no-check-certificate 'https://drive.google.com/uc?export=download&id=1OIU9ICMebvZXJ0Grc2SLlMep3x9EkZtz' -O perashki.txt
!wget -qq --no-check-certificate 'https://drive.google.com/uc?export=download&id=1v66uAEKL3KunyylYitNKggdl2gCeYgZZ' -O poroshki.txt
!git clone https://github.com/UniversalDependencies/UD_Russian-SynTagRus.git
!wget -qq https://raw.githubusercontent.com/DanAnastasyev/neuromorphy/master/neuromorphy/train/corpus_iterator.py

fatal: destination path 'UD_Russian-SynTagRus' already exists and is not an empty directory.


In [0]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if torch.cuda.is_available():
    from torch.cuda import FloatTensor, LongTensor
    DEVICE = torch.device('cuda')
else:
    from torch import FloatTensor, LongTensor
    DEVICE = torch.device('cpu')

np.random.seed(42)

# Word-Level Text Generation

Сегодня занимаемся, в основном, тем, что генерируем *пирожки* и *порошки*.

*(Данные без спросу скачаны с сайта http://poetory.ru)*

Пирожки - это вот:

In [30]:
!head perashki.txt

старик вытягивает сети
они пусты и лишь в конце
записка рыба недоступна
или вне действия сети

олег адепт шизофрении
шагает бодро из окна
и жызнь летит перед глазами
да не одна а сразу две



Порошки вот:

In [31]:
!head poroshki.txt

кто любит цоя кто покушать
кто на рассвете пенье птиц
а я люблю в коротких платьях
физлиц

твой монолог так гениален
что я чуть не открыла дверь
но станиславский тихо сверху
не верь



Не перепутайте!

Вообще, пирожок - это четверостишие, написанное четырехстопным ямбом по схеме 9-8-9-8. У порошка схема 9-8-9-2.

In [0]:
vowels = 'ёуеыаоэяию'

odd_pattern = '-+-+-+-+-'
even_pattern = '-+-+-+-+'

Считываем данные:

In [0]:
def read_poem(path):
    poem = []
    with open(path, encoding='utf8') as f:
        for line in f:
            line = line.rstrip()
            if len(line) == 0:
                yield poem
                poem = []
                continue
            
            poem.extend(line.split() + ['\\n'])
            
perashki = list(read_poem('perashki.txt'))
poroshki = list(read_poem('poroshki.txt'))

Построим датасет для порошков:

In [7]:
from torchtext.data import Field, Example, Dataset, BucketIterator

text_field = Field(init_token='<s>', eos_token='</s>')
        
fields = [('text', text_field)]
examples = [Example.fromlist([poem], fields) for poem in poroshki]
dataset = Dataset(examples, fields)

text_field.build_vocab(dataset, min_freq=7)

print('Vocab size =', len(text_field.vocab))
train_dataset, test_dataset = dataset.split(split_ratio=0.9)

train_iter, test_iter = BucketIterator.splits(datasets=(train_dataset, test_dataset), batch_sizes=(32, 128), 
                                              shuffle=True, device=DEVICE, sort=False)

Vocab size = 6298


**Задание** Напишите класс языковой модели.

In [0]:
class LMModel(nn.Module):
    def __init__(self, vocab_size, emb_dim=256, lstm_hidden_dim=256, num_layers=1):
        super().__init__()

        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.LSTM(input_size=emb_dim, hidden_size=lstm_hidden_dim)
        
        self._out_layer = nn.Linear(lstm_hidden_dim, vocab_size)
        
        self._init_weights()

    def _init_weights(self, init_range=0.1):
        self._emb.weight.data.uniform_(-init_range, init_range)
        self._out_layer.bias.data.zero_()
        self._out_layer.weight.data.uniform_(-init_range, init_range)

    def forward(self, inputs, hidden=None):
        embs = self._emb(inputs)
        output, hidden = self._rnn(embs, hidden)
        output = self._out_layer(output)
        return output, hidden

In [0]:
batch = next(iter(train_iter))

In [10]:
model = LMModel(vocab_size=len(train_iter.dataset.fields['text'].vocab)).to(DEVICE)

model(batch.text)

(tensor([[[ 0.0251, -0.0173,  0.0144,  ...,  0.0027, -0.0017,  0.0269],
          [ 0.0251, -0.0173,  0.0144,  ...,  0.0027, -0.0017,  0.0269],
          [ 0.0251, -0.0173,  0.0144,  ...,  0.0027, -0.0017,  0.0269],
          ...,
          [ 0.0251, -0.0173,  0.0144,  ...,  0.0027, -0.0017,  0.0269],
          [ 0.0251, -0.0173,  0.0144,  ...,  0.0027, -0.0017,  0.0269],
          [ 0.0251, -0.0173,  0.0144,  ...,  0.0027, -0.0017,  0.0269]],
 
         [[ 0.0423, -0.0151,  0.0136,  ...,  0.0002, -0.0152,  0.0473],
          [ 0.0144, -0.0198,  0.0121,  ...,  0.0039, -0.0109,  0.0292],
          [ 0.0181, -0.0206,  0.0079,  ..., -0.0143, -0.0073,  0.0349],
          ...,
          [ 0.0201, -0.0149,  0.0221,  ...,  0.0041, -0.0131,  0.0272],
          [ 0.0144, -0.0198,  0.0121,  ...,  0.0039, -0.0109,  0.0292],
          [ 0.0164, -0.0234,  0.0136,  ...,  0.0071, -0.0102,  0.0373]],
 
         [[ 0.0378, -0.0156,  0.0098,  ...,  0.0131, -0.0087,  0.0434],
          [ 0.0105, -0.0214,

**Задание** Добавьте подсчет потерей с маскингом паддингов.

In [0]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(model, criterion, data_iter, unk_idx, pad_idx, optimizer=None, name=None):
    epoch_loss = 0
    
    is_train = not optimizer is None
    name = name or ''
    model.train(is_train)
    
    batches_count = len(data_iter)
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):                
                logits, _ = model(batch.text)

                targets = torch.cat((batch.text[1:], batch.text.new_ones((1, batch.text.shape[1]))))

                loss = criterion(logits.view(-1, logits.shape[-1]), targets.view(-1))
                
                epoch_loss += loss.item()

                if optimizer:
                    optimizer.zero_grad()
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), 1.)
                    optimizer.step()

                progress_bar.update()
                progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(name, loss.item(), 
                                                                                         math.exp(loss.item())))
                
            progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(
                name, epoch_loss / batches_count, math.exp(epoch_loss / batches_count))
            )
            progress_bar.refresh()

    return epoch_loss / batches_count


def fit(model, criterion, optimizer, train_iter, epochs_count=1, unk_idx=0, pad_idx=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        train_loss = do_epoch(model, criterion, train_iter, unk_idx, pad_idx, optimizer, name_prefix + 'Train:')
        
        if not val_iter is None:
            val_loss = do_epoch(model, criterion, val_iter, unk_idx, pad_idx, None, name_prefix + '  Val:')
            
            if best_val_loss and val_loss > best_val_loss:
                optimizer.param_groups[0]['lr'] /= 4.
                print('Optimizer lr = {:g}'.format(optimizer.param_groups[0]['lr']))
            else:
                best_val_loss = val_loss
        print()
        generate(model)
        print()

**Задание** Напишите функцию-генератор для модели.

In [12]:
def sample(probs, temp):
    probs = F.log_softmax(probs.squeeze(), dim=0)
    probs = (probs / temp).exp()
    probs /= probs.sum()
    probs = probs.cpu().numpy()

    return np.random.choice(np.arange(len(probs)), p=probs)


def generate(model, temp=0.6):
    model.eval()
    with torch.no_grad():        
        prev_token = train_iter.dataset.fields['text'].vocab.stoi['<s>']
        end_token = train_iter.dataset.fields['text'].vocab.stoi['</s>']
        
        hidden = None
        for _ in range(150):
            probs, hidden = model(LongTensor([[prev_token]]), hidden)
            prev_token = sample(probs, temp)
            
            if prev_token == end_token:
                return
            
            if train_iter.dataset.fields['text'].vocab.itos[prev_token] == '\\n':
                print()
            else:
                print(train_iter.dataset.fields['text'].vocab.itos[prev_token], end=' ')
                
generate(model)

изольда разбег тоскливо девчонки пошёл пошёл счастья влезает жалко палату стал сталин сусанин бес одним сергей навсегда толпою часов стены лучах ад страшный бюст надеюсь мин полно поцелуй визг своей кобзон ван жене пункт смены понты намного ай ищешь выбираю суть зачат прошу лягу торт будьте толпа слышен икры уста спустился ключи половиной кабинета далеко несите земля дно слова мяса сидела пришлось состоит растёт искать дыханье лень пинок довольно фома кабинет пей был пушистый отверг страстно искры сердца спорить вроде вернее останки речь тому страха морковь сук связала жёлтый список кричат порей доска подарили исаев жён строчки спас год ногами конечно рухнул эс осколки лупит забыв кляня грека узнал народ сиськи пух отправил пониже простить георгий пломбир выйти изменила грусти получился капкан голов борща ставит спальни ремня букетик кто пирог одеяло дверью кричат бок похожа кухню сантехник ножки яйцо стопарь скажыте ничей кактус змея окон лежит кружок менять пальто дрожа 

In [13]:
model = LMModel(vocab_size=len(train_iter.dataset.fields['text'].vocab)).to(DEVICE)

pad_idx = train_iter.dataset.fields['text'].vocab.stoi['<pad>']
unk_idx = train_iter.dataset.fields['text'].vocab.stoi['<unk>']
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx).to(DEVICE)

optimizer = optim.SGD(model.parameters(), lr=20., weight_decay=1e-6)

fit(model, criterion, optimizer, train_iter, epochs_count=30, unk_idx=unk_idx, pad_idx=pad_idx, val_iter=test_iter)

[1 / 30]   Val: Loss = 4.58895, PPX = 98.39: 100%|██████████| 19/19 [00:00<00:00, 44.11it/s]



<unk> <unk> <unk> <unk> 
<unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> 

[2 / 30] Train: Loss = 4.49331, PPX = 89.42:   0%|          | 3/677 [00:00<00:22, 29.99it/s] 

<unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> 
<unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> 


[3 / 30] Train: Loss = 4.81527, PPX = 123.38:   1%|          | 4/677 [00:00<00:19, 35.15it/s]

...

Optimizer lr = 1.16415e-09

я <unk> <unk> 
<unk> <unk> <unk> 
и <unk> <unk> 
<unk> 



[30 / 30] Train: Loss = 3.28454, PPX = 26.70:   1%|          | 5/677 [00:00<00:22, 30.00it/s]

Optimizer lr = 2.91038e-10

<unk> <unk> 
на <unk> <unk> 
и <unk> <unk> 
<unk> 



[30 / 30]   Val: Loss = 3.69627, PPX = 40.30: 100%|██████████| 19/19 [00:00<00:00, 47.33it/s]


Optimizer lr = 7.27596e-11

когда <unk> <unk> 
в <unk> <unk> 
<unk> <unk> <unk> 
<unk> 



**Задание** Добавьте маскинг `<unk>` токенов при тренировке модели.

In [0]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(model, criterion, data_iter, unk_idx, pad_idx, optimizer=None, name=None):
    epoch_loss = 0
    
    is_train = not optimizer is None
    name = name or ''
    model.train(is_train)
    
    batches_count = len(data_iter)
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):                
                logits, _ = model(batch.text)

                targets = torch.cat((batch.text[1:], batch.text.new_ones((1, batch.text.shape[1])))).view(-1)

                loss = criterion(logits.view(-1, logits.shape[-1]), targets)
                
                mask = (1 - ((targets == unk_idx) + (targets == pad_idx))).float()
                
                loss = (loss * mask).sum() / mask.sum()
                
                epoch_loss += loss.item()

                if optimizer:
                    optimizer.zero_grad()
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), 1.)
                    optimizer.step()

                progress_bar.update()
                progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(name, loss.item(), 
                                                                                         math.exp(loss.item())))
                
            progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(
                name, epoch_loss / batches_count, math.exp(epoch_loss / batches_count))
            )
            progress_bar.refresh()

    return epoch_loss / batches_count


def fit(model, criterion, optimizer, train_iter, epochs_count=1, unk_idx=0, pad_idx=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        train_loss = do_epoch(model, criterion, train_iter, unk_idx, pad_idx, optimizer, name_prefix + 'Train:')
        
        if not val_iter is None:
            val_loss = do_epoch(model, criterion, val_iter, unk_idx, pad_idx, None, name_prefix + '  Val:')
            
            if best_val_loss and val_loss > best_val_loss:
                optimizer.param_groups[0]['lr'] /= 4.
                print('Optimizer lr = {:g}'.format(optimizer.param_groups[0]['lr']))
            else:
                best_val_loss = val_loss
        print()
        generate(model)
        print()

In [15]:
model = LMModel(vocab_size=len(train_iter.dataset.fields['text'].vocab)).to(DEVICE)

pad_idx = train_iter.dataset.fields['text'].vocab.stoi['<pad>']
unk_idx = train_iter.dataset.fields['text'].vocab.stoi['<unk>']
criterion = nn.CrossEntropyLoss(reduction='none').to(DEVICE)

optimizer = optim.SGD(model.parameters(), lr=20., weight_decay=1e-6)

fit(model, criterion, optimizer, train_iter, epochs_count=30, unk_idx=unk_idx, pad_idx=pad_idx, val_iter=test_iter)

[2 / 30] Train: Loss = 4.32521, PPX = 75.58:   1%|          | 4/677 [00:00<00:20, 33.33it/s]


я из день четыре с тобою 
я не могу и не с собой 
я так с себе что не думать 
в них и 



[3 / 30] Train: Loss = 3.98990, PPX = 54.05:   1%|          | 4/677 [00:00<00:24, 27.61it/s]


в связи я своей не может 
не потому что то я 
а я как не по эту 
и ты 



[4 / 30] Train: Loss = 4.24416, PPX = 69.70:   1%|          | 4/677 [00:00<00:22, 29.81it/s]

...

Optimizer lr = 1.13687e-12

в лесу ему как будто ночью 
что не умел как бог бог 
но каждый раз уже оксану 
и ест 



[30 / 30] Train: Loss = 3.36206, PPX = 28.85:   1%|          | 4/677 [00:00<00:20, 33.28it/s]

Optimizer lr = 2.84217e-13

чтоб не могу понять собою 
и что на самом деле нет 
а вы не с ним с кем мы в нем больше 
помочь 



[30 / 30]   Val: Loss = 4.12915, PPX = 62.13: 100%|██████████| 19/19 [00:00<00:00, 45.96it/s]


Optimizer lr = 7.10543e-14

я не могу понять деревня 
и как ни больше не а тем 
я с ней в кровать с любовью и не видно 
ни в чём 



## Улучшаем модель

### Tying input and output embeddings

В модели есть два эмбеддинга - входной и выходной. Красивая и полезная в жизни идея - учить только одну матрицу, расшаренную между ними: [Using the Output Embedding to Improve Language Models](http://www.aclweb.org/anthology/E17-2025).

От идеи одни плюсы: получается намного меньше обучаемых параметров и при этом достаточно заметно более высокое качество.

**Задание** Реализуйте это. Достаточно написать что-то типа этого в конструкторе:

`self._out_layer.weight = self._emb.weight`

In [0]:
class InputOutputLMModel(nn.Module):
    def __init__(self, vocab_size, emb_dim=512, lstm_hidden_dim=512, num_layers=1):
        super().__init__()

        self._dropout = nn.Dropout(0.4)
        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.LSTM(input_size=emb_dim, hidden_size=lstm_hidden_dim, num_layers=num_layers)
        
        self._out_layer = nn.Linear(lstm_hidden_dim, vocab_size)
        self._out_layer.weight = self._emb.weight
        
        self._init_weights()

    def _init_weights(self, init_range=0.1):
        self._emb.weight.data.uniform_(-init_range, init_range)
        self._out_layer.bias.data.zero_()
        self._out_layer.weight.data.uniform_(-init_range, init_range)

    def forward(self, inputs, hidden=None):
        embs = self._emb(inputs)
        output, hidden = self._rnn(embs, hidden)
        return self._out_layer(self._dropout(output)), hidden

In [17]:
model = InputOutputLMModel(vocab_size=len(train_iter.dataset.fields['text'].vocab)).to(DEVICE)

pad_idx = train_iter.dataset.fields['text'].vocab.stoi['<pad>']
unk_idx = train_iter.dataset.fields['text'].vocab.stoi['<unk>']
criterion = nn.CrossEntropyLoss(reduction='none').to(DEVICE)

optimizer = optim.SGD(model.parameters(), lr=20., weight_decay=1e-6)

fit(model, criterion, optimizer, train_iter, epochs_count=30, unk_idx=unk_idx, pad_idx=pad_idx, val_iter=test_iter)

[2 / 30] Train: Loss = 4.47596, PPX = 87.88:   0%|          | 3/677 [00:00<00:28, 23.45it/s]


как по виде то в мире 
и все в него и не сказал 
а вот уже не под 
ничо 



[3 / 30] Train: Loss = 4.32368, PPX = 75.47:   0%|          | 2/677 [00:00<00:33, 20.29it/s]

Optimizer lr = 5

олег на рынке и очень 
у вас на все на всех на это 
а я не раз на ней о том да 
не ссы 



[4 / 30] Train: Loss = 4.14442, PPX = 63.08:   0%|          | 3/677 [00:00<00:29, 23.20it/s]

...

Optimizer lr = 7.27596e-11

я видел двух тебя как будто 
но в самом деле всё в стране 
и я не знаю что не видел 
в аду 



[30 / 30] Train: Loss = 3.42859, PPX = 30.83:   0%|          | 3/677 [00:00<00:29, 23.12it/s]

Optimizer lr = 1.81899e-11

мне не нужна на этом свете 
уже три года на год 
и в этом мире он не в силах 
не спит 



[30 / 30]   Val: Loss = 4.04218, PPX = 56.95: 100%|██████████| 19/19 [00:00<00:00, 32.08it/s]


Optimizer lr = 4.54747e-12

из всех сторон и я влюбилась 
что был один со мной как ты 
а у меня там на краю 
жену 



### Добавление информации в выборку

Сейчас у нас каждое слово предствляется одним индексом. Модели очень сложно узнать, сколько в нем слогов - а значит, сложно генерировать корректное стихотворение.

На самом деле к каждому слову можно приписать кусочек из метрического шаблона:

![](https://hsto.org/web/59a/b39/bd0/59ab39bd020c49a78a12cbab62c80181.png =x200)

**Задание** Обновите функцию `read_poem`, пусть она генерирует два списка - список слов и список кусков шаблона.  
Добавьте в модель вход - последовательности шаблонов, конкатенируйте их эмбеддинги со словами.  
Дополнительная идея - заставьте модель угадывать, какой шаблон должен идти следующим (где-то половина будет подходящими, остальные - нет). Добавьте дополнительные потери от угадывания шаблона.

In [0]:
def read_poem(path):
    poem, patterns = [], []
    with open(path, encoding='utf8') as f:
        for line in f:
            line = line.rstrip()
            if len(line) == 0:
                yield poem, patterns
                poem, patterns = [], []
                continue

            pattern = []
            template = ''
            sign = '-'
            
            for word in line.split():
                for char in word:
                    if char in vowels:
                        template += sign
                        sign = '+'if sign == '-' else '-'
                pattern.append(template)
                template = ''

            poem.extend(line.split() + ['\\n'])
            patterns.extend(pattern + ['\\n'])

poroshki = np.array(list(read_poem('poroshki.txt')))

In [0]:
poroshki, poroshki_patterns = poroshki[:, 0], poroshki[:, 1]

In [20]:
from torchtext.data import Field, Example, Dataset, BucketIterator

text_field = Field(init_token='<s>', eos_token='</s>')
pattern_field = Field(init_token='<s>', eos_token='</s>')
        
fields = [('text', text_field), ('pattern', pattern_field)]
examples = [Example.fromlist([poem, pattern], fields) for poem, pattern in zip(poroshki, poroshki_patterns)]
dataset = Dataset(examples, fields)

text_field.build_vocab(dataset, min_freq=7)
pattern_field.build_vocab(dataset)

print('Vocab size =', len(text_field.vocab))
train_dataset, test_dataset = dataset.split(split_ratio=0.9)

train_iter, test_iter = BucketIterator.splits(datasets=(train_dataset, test_dataset), batch_sizes=(32, 128), 
                                              shuffle=True, device=DEVICE, sort=False)

Vocab size = 6298


In [0]:
class PatternModel(nn.Module):
    def __init__(self, vocab_size, pattern_size, emb_dim=512, lstm_hidden_dim=512, num_layers=1):
        super().__init__()

        self._dropout = nn.Dropout(0.4)
        self._text_emb = nn.Embedding(vocab_size, emb_dim)
        self._pattern_emb = torch.eye(pattern_size).cuda()
        self._rnn = nn.LSTM(input_size=emb_dim+pattern_size, hidden_size=lstm_hidden_dim, num_layers=num_layers)
        
        self._text_out_layer = nn.Linear(lstm_hidden_dim, vocab_size)
        self._text_out_layer.weight = self._text_emb.weight
        
        self._pattern_out_layer = nn.Linear(lstm_hidden_dim, pattern_size)
        
        self._init_weights()

    def _init_weights(self, init_range=0.1):
        self._text_emb.weight.data.uniform_(-init_range, init_range)
        self._text_out_layer.bias.data.zero_()
        self._text_out_layer.weight.data.uniform_(-init_range, init_range)

    def forward(self, inputs, patterns, hidden=None):
        text_embs = self._text_emb(inputs)
        
        pattern_embs = self._pattern_emb[patterns]
        
        embs = torch.cat((text_embs, pattern_embs), -1)
        
        output, hidden = self._rnn(embs, hidden)
        
        text_output = self._text_out_layer(self._dropout(output))
        pattern_output = self._pattern_out_layer(self._dropout(output))
        
        return text_output, pattern_output, hidden

In [0]:
model = PatternModel(vocab_size=len(train_iter.dataset.fields['text'].vocab),
                     pattern_size=len(train_iter.dataset.fields['pattern'].vocab)).to(DEVICE)

In [23]:
def sample(probs, temp):
    probs = F.log_softmax(probs.squeeze(), dim=0)
    probs = (probs / temp).exp()
    probs /= probs.sum()
    probs = probs.cpu().numpy()

    return np.random.choice(np.arange(len(probs)), p=probs)

def generate(model, temp=0.6):
    model.eval()
    with torch.no_grad():        
        prev_text = train_iter.dataset.fields['text'].vocab.stoi['<s>']
        prev_pattern = train_iter.dataset.fields['pattern'].vocab.stoi['<s>']
        end_text = train_iter.dataset.fields['text'].vocab.stoi['</s>']
        end_pattern = train_iter.dataset.fields['pattern'].vocab.stoi['</s>']
        
        hidden = None
        sign = '-'
        for _ in range(150):
            text_probs, _, hidden = model(LongTensor([[prev_text]]), LongTensor([[prev_pattern]]), hidden)
            prev_text = sample(text_probs, temp)
            
            pattern = ''
            for char in train_iter.dataset.fields['text'].vocab.itos[prev_text]:
                if char in vowels:
                    pattern += sign
                    sign = '+' if sign == '-' else '-'
            prev_pattern = train_iter.dataset.fields['pattern'].vocab.stoi[pattern]
            
            if prev_text == end_text:
                return
            
            if train_iter.dataset.fields['text'].vocab.itos[prev_text] == '\\n':
                sign = '-'
                print()
            else:
                print(train_iter.dataset.fields['text'].vocab.itos[prev_text], end=' ')
                
generate(model)

год столу рифма вырос назад иной сцену дега игру ан ребёнок хвосты молчанье коллеги какого вашем пора настал сложив поймать солнце четыре сдох краю друг грим обнимку рыбка кормит горы роняя штирлиц зайка закричал смотря уроды пытаюсь седьмой скрипит превратился вслух догадался дала меняет догадался достиг съезде выход камней ужас блин увы дают покоя собакой одетый крыш украл ум дык пера вылез времён шаги держали опытный режим необходимо такого фу семён теребя дурака слезу сложив любые светится бывают высоты навстречу небрит крики обычным шаинский гуляш прах будешь полна тычет ага вертолёт богатство коту плывёт вине достиг пыл плоть ева суёт вин тыгдым жива снилось жрал слове рояля сидел трудом байкал взор шага изпод летит древних каждого первых песнь обязан глеб тыкать козлом седин лез ежа добавить судья побыть дождя старушка рыдая сказал эркюль даче сознанья холодильник оксана чехов сюрприз юности таким выпь зелёных баню зажал чуковский машине ильич вова стакан 

In [0]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(model, criterion, data_iter, unk_idx, pad_idx, optimizer=None, name=None):
    epoch_loss = 0
    
    is_train = not optimizer is None
    name = name or ''
    model.train(is_train)
    
    batches_count = len(data_iter)
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):
                text_logits, pattern_logits, _ = model(batch.text, batch.pattern)

                text_targets = torch.cat((batch.text[1:], batch.text.new_ones((1, batch.text.shape[1])))).view(-1)
                
                additional = None
                if optimizer:
                    pattern_targets = torch.cat((batch.pattern[1:], batch.pattern.new_ones((1, batch.pattern.shape[1])))).view(-1)
                    pattern_loss = criterion(pattern_logits.view(-1, pattern_logits.shape[-1]), pattern_targets)
                    pattern_mask = (1 - (pattern_targets == pad_idx)).float()
                    additional = (pattern_loss * pattern_mask).sum() / pattern_mask.sum()

                text_loss = criterion(text_logits.view(-1, text_logits.shape[-1]), text_targets)
                
                text_mask = (1 - ((text_targets == unk_idx) + (text_targets == pad_idx))).float()
                
                loss = (text_loss * text_mask).sum() / text_mask.sum()
                if additional:
                    loss += additional
                
                epoch_loss += loss.item()

                if optimizer:
                    optimizer.zero_grad()
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), 1.)
                    optimizer.step()

                progress_bar.update()
                progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(name, loss.item(), 
                                                                                         math.exp(loss.item())))
                
            progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(
                name, epoch_loss / batches_count, math.exp(epoch_loss / batches_count))
            )
            progress_bar.refresh()

    return epoch_loss / batches_count


def fit(model, criterion, optimizer, train_iter, epochs_count=1, unk_idx=0, pad_idx=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        train_loss = do_epoch(model, criterion, train_iter, unk_idx, pad_idx, optimizer, name_prefix + 'Train:')
        
        if not val_iter is None:
            val_loss = do_epoch(model, criterion, val_iter, unk_idx, pad_idx, None, name_prefix + '  Val:')
            
            if best_val_loss and val_loss > best_val_loss:
                optimizer.param_groups[0]['lr'] /= 4.
                print('Optimizer lr = {:g}'.format(optimizer.param_groups[0]['lr']))
            else:
                best_val_loss = val_loss
        print()
        generate(model)
        print()

In [25]:
model = PatternModel(vocab_size=len(train_iter.dataset.fields['text'].vocab),
                     pattern_size=len(train_iter.dataset.fields['pattern'].vocab)).to(DEVICE)

pad_idx = train_iter.dataset.fields['text'].vocab.stoi['<pad>']
unk_idx = train_iter.dataset.fields['text'].vocab.stoi['<unk>']
criterion = nn.CrossEntropyLoss(reduction='none').to(DEVICE)

optimizer = optim.SGD(model.parameters(), lr=20., weight_decay=1e-6)

fit(model, criterion, optimizer, train_iter, epochs_count=30, unk_idx=unk_idx, pad_idx=pad_idx, val_iter=test_iter)

[2 / 30] Train: Loss = 6.74021, PPX = 845.74:   0%|          | 2/677 [00:00<00:35, 19.02it/s]


на день и не не знаю было 
о вдруг в вас не тебя я 
и не 



[3 / 30] Train: Loss = 6.45594, PPX = 636.47:   0%|          | 2/677 [00:00<00:36, 18.55it/s]


на самом деле на работу 
и не могу тогда не друг 
и жизнь не все не буду 
не те 



[4 / 30] Train: Loss = 5.49055, PPX = 242.39:   0%|          | 2/677 [00:00<00:35, 19.08it/s]

...

Optimizer lr = 4.65661e-09

в одном из нас в деревне нету 
лежит в глазах и в шалаше 
а ты всё мной в меня сегодня 
как раз 



[30 / 30] Train: Loss = 4.35838, PPX = 78.13:   0%|          | 1/677 [00:00<00:51, 13.02it/s]

Optimizer lr = 1.16415e-09

я вас люблю тебя родная 
я в этот раз свою весь день 
и что поделать с ними было 
как все 



[30 / 30]   Val: Loss = 3.87039, PPX = 47.96: 100%|██████████| 19/19 [00:00<00:00, 30.78it/s]


Optimizer lr = 2.91038e-10

я не люблю тебя до бабы 
что к нам и в рай не в шалаше 
а не могли бы вы мне стали 
в раю 



### Увеличиваем выборку

У нас есть выборка для пирожков, которая заметно больше.

**Задание** Обучитесь на ней.

In [0]:
def read_poem(path):
    poem = []
    with open(path, encoding='utf8') as f:
        for line in f:
            line = line.rstrip()
            if len(line) == 0:
                yield poem
                poem = []
                continue
            
            poem.extend(line.split() + ['\\n'])
            
perashki = list(read_poem('perashki.txt'))
poroshki = list(read_poem('poroshki.txt'))

In [7]:
from torchtext.data import Field, Example, Dataset, BucketIterator

text_field = Field(init_token='<s>', eos_token='</s>')
        
fields = [('text', text_field)]
examples = [Example.fromlist([poem], fields) for poem in perashki]
dataset = Dataset(examples, fields)

text_field.build_vocab(dataset, min_freq=7)

print('Vocab size =', len(text_field.vocab))
train_dataset, test_dataset = dataset.split(split_ratio=0.9)

train_iter, test_iter = BucketIterator.splits(datasets=(train_dataset, test_dataset), batch_sizes=(32, 128), 
                                              shuffle=True, device=DEVICE, sort=False)

Vocab size = 13070


In [0]:
def sample(probs, temp):
    probs = F.log_softmax(probs.squeeze(), dim=0)
    probs = (probs / temp).exp()
    probs /= probs.sum()
    probs = probs.cpu().numpy()

    return np.random.choice(np.arange(len(probs)), p=probs)


def generate(model, temp=0.6):
    model.eval()
    with torch.no_grad():        
        prev_token = train_iter.dataset.fields['text'].vocab.stoi['<s>']
        end_token = train_iter.dataset.fields['text'].vocab.stoi['</s>']
        
        hidden = None
        for _ in range(150):
            probs, hidden = model(LongTensor([[prev_token]]), hidden)
            prev_token = sample(probs, temp)
            
            if prev_token == end_token:
                return
            
            if train_iter.dataset.fields['text'].vocab.itos[prev_token] == '\\n':
                print()
            else:
                print(train_iter.dataset.fields['text'].vocab.itos[prev_token], end=' ')

In [0]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(model, criterion, data_iter, unk_idx, pad_idx, optimizer=None, name=None):
    epoch_loss = 0
    
    is_train = not optimizer is None
    name = name or ''
    model.train(is_train)
    
    batches_count = len(data_iter)
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):                
                logits, _ = model(batch.text)

                targets = torch.cat((batch.text[1:], batch.text.new_ones((1, batch.text.shape[1])))).view(-1)

                loss = criterion(logits.view(-1, logits.shape[-1]), targets)
                
                mask = (1 - ((targets == unk_idx) + (targets == pad_idx))).float()
                
                loss = (loss * mask).sum() / mask.sum()
                
                epoch_loss += loss.item()

                if optimizer:
                    optimizer.zero_grad()
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), 1.)
                    optimizer.step()

                progress_bar.update()
                progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(name, loss.item(), 
                                                                                         math.exp(loss.item())))
                
            progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(
                name, epoch_loss / batches_count, math.exp(epoch_loss / batches_count))
            )
            progress_bar.refresh()

    return epoch_loss / batches_count


def fit(model, criterion, optimizer, train_iter, epochs_count=1, unk_idx=0, pad_idx=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        train_loss = do_epoch(model, criterion, train_iter, unk_idx, pad_idx, optimizer, name_prefix + 'Train:')
        
        if not val_iter is None:
            val_loss = do_epoch(model, criterion, val_iter, unk_idx, pad_idx, None, name_prefix + '  Val:')
            
            if best_val_loss and val_loss > best_val_loss:
                optimizer.param_groups[0]['lr'] /= 4.
                print('Optimizer lr = {:g}'.format(optimizer.param_groups[0]['lr']))
            else:
                best_val_loss = val_loss
        print()
        generate(model)
        print()

In [0]:
class InputOutputLMModel(nn.Module):
    def __init__(self, vocab_size, emb_dim=512, lstm_hidden_dim=512, num_layers=1):
        super().__init__()

        self._dropout = nn.Dropout(0.4)
        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.LSTM(input_size=emb_dim, hidden_size=lstm_hidden_dim, num_layers=num_layers)
        
        self._out_layer = nn.Linear(lstm_hidden_dim, vocab_size)
        self._out_layer.weight = self._emb.weight
        
        self._init_weights()

    def _init_weights(self, init_range=0.1):
        self._emb.weight.data.uniform_(-init_range, init_range)
        self._out_layer.bias.data.zero_()
        self._out_layer.weight.data.uniform_(-init_range, init_range)

    def forward(self, inputs, hidden=None):
        embs = self._emb(inputs)
        output, hidden = self._rnn(embs, hidden)
        return self._out_layer(self._dropout(output)), hidden

In [11]:
model = InputOutputLMModel(vocab_size=len(train_iter.dataset.fields['text'].vocab)).to(DEVICE)

pad_idx = train_iter.dataset.fields['text'].vocab.stoi['<pad>']
unk_idx = train_iter.dataset.fields['text'].vocab.stoi['<unk>']
criterion = nn.CrossEntropyLoss(reduction='none').to(DEVICE)

optimizer = optim.SGD(model.parameters(), lr=20., weight_decay=1e-6)

fit(model, criterion, optimizer, train_iter, epochs_count=15, unk_idx=unk_idx, pad_idx=pad_idx, val_iter=test_iter)

  0%|          | 0/1386 [00:00<?, ?it/s]


я помню но не так и не серёжа 
а в толпе ни в них не так 
а я не так и не было 
и не один и не в меня 



  0%|          | 1/1386 [00:00<02:48,  8.24it/s]


олег не с николаем в бога 
и утром в понедельник и 
на ночь висит на стенах месте 
а в жизни в жизни не смотреть 



  0%|          | 0/1386 [00:00<?, ?it/s]

...


я так люблю тебя родная 
что даже к смерти и в метро 
и в ванной в зеркало да в небо 
а ты как будто бы не так 



[15 / 15] Train: Loss = 3.15733, PPX = 23.51:   0%|          | 1/1386 [00:00<02:42,  8.50it/s]

Optimizer lr = 0.3125

я умер а пока что умер 
а я уже не осознал 
как ты и знаешь что не любишь 
и как меня не узнаю 



[15 / 15]   Val: Loss = 4.16203, PPX = 64.20: 100%|██████████| 39/39 [00:01<00:00, 20.06it/s]


Optimizer lr = 0.078125

когда мы с пушкиным бежали 
в столице на меня смотреть 
я в этот день не представляю 
а в принципе я не хочу 



### Transfer learning

Простой и приятный способ улучшения модели - сделать перенос обученной на большом корпусе модели на меньшего объема датасет.

Популярен этот способ больше в компьютерном зрении: [Transfer learning, cs231n](http://cs231n.github.io/transfer-learning/) - там есть огромный ImageNet, на котором предобучают модель, чтобы потом заморозить нижние слои и заменить выходные. В итоге модель использует универсальные представления данных, выученные на большом корпусе, но для предсказания совсем других меток - и качество очень здорово растет.

Нам такие извращения пока не нужны (хотя потом пригодятся, ключевые слова: ULMFiT, ELMo и компания). Просто возьмем обученную на большем корпусе модель и поучим ее на меньшем корпусе. Ей всего-то нужно новый матрический шаблон последней строки выучить.

**Задание** Обученную в прошлом пункте модель дообучите на порошки.

In [0]:
from torchtext.data import Field, Example, Dataset, BucketIterator
        
fields = [('text', text_field)]
examples = [Example.fromlist([poem], fields) for poem in poroshki]
dataset = Dataset(examples, fields)

train_dataset, test_dataset = dataset.split(split_ratio=0.9)

train_iter, test_iter = BucketIterator.splits(datasets=(train_dataset, test_dataset), batch_sizes=(32, 128), 
                                              shuffle=True, device=DEVICE, sort=False)

In [13]:
fit(model, criterion, optimizer, train_iter, epochs_count=30, unk_idx=unk_idx, pad_idx=pad_idx, val_iter=test_iter)

[2 / 30] Train: Loss = 4.59046, PPX = 98.54:   0%|          | 2/677 [00:00<00:38, 17.40it/s] 


олег не умер это умер 
но не к тебе а в смысле я 
а то ли на людей всё то же 
теперь 



[3 / 30] Train: Loss = 4.47910, PPX = 88.16:   0%|          | 2/677 [00:00<00:40, 16.65it/s]


я вел себя на жабры с пивом 
и с каждым годом все с пятном 
и с каждой строчкой люди ближе 
и ходят голые стволы 



[4 / 30] Train: Loss = 4.53873, PPX = 93.57:   0%|          | 2/677 [00:00<00:40, 16.58it/s] 

...


когда ты будешь спать с порога 
я буду всё на свете зла 
но я не стала и с собою 
и в воду 



[30 / 30] Train: Loss = 4.10489, PPX = 60.64:   0%|          | 2/677 [00:00<00:41, 16.36it/s]


я по утрам хожу в деревне 
до самой смерти где 
теперь ты в нём же в рот не знаешь 
но даже им не в шутку 



[30 / 30]   Val: Loss = 4.16396, PPX = 64.33: 100%|██████████| 19/19 [00:00<00:00, 20.81it/s]



я не могу тебе на свете 
не только в теле никогда 
и только ты не отвечаешь 
в пивной 



### Conditional language model

Ещё лучше - просто учиться на обоих корпусах сразу. Объедините пирожки и порошки, для каждого храните индекс 0/1 - был ли это пирожок или порошок. Добавьте вход - этот индекс и конкатенируйте его либо к каждому эмбеддингу слов, либо к каждому выходу из LSTM.

**Задание** Научите единую модель, у которой можно просить сгенерировать пирожок или порошок.

In [0]:
def read_poem(path):
    poem = []
    with open(path, encoding='utf8') as f:
        for line in f:
            line = line.rstrip()
            if len(line) == 0:
                yield poem
                poem = []
                continue
            
            poem.extend(line.split() + ['\\n'])
            
perashki = list(read_poem('perashki.txt'))
poroshki = list(read_poem('poroshki.txt'))

In [0]:
labels = [0] * len(perashki) + [1] * len(poroshki)
data = perashki + poroshki

In [35]:
from torchtext.data import Field, Example, Dataset, BucketIterator

text_field = Field(init_token='<s>', eos_token='</s>')
label_field = Field(sequential=False)
        
fields = [('text', text_field), ('label', label_field)]
examples = [Example.fromlist([poem, label], fields) for poem, label in zip(data, labels)]
dataset = Dataset(examples, fields)

text_field.build_vocab(dataset, min_freq=7)
label_field.build_vocab(dataset)

print('Text vocab size =', len(text_field.vocab))
print('Label vocab size =', len(label_field.vocab))
train_dataset, test_dataset = dataset.split(split_ratio=0.9)

train_iter, test_iter = BucketIterator.splits(datasets=(train_dataset, test_dataset), batch_sizes=(32, 128), 
                                              shuffle=True, device=DEVICE, sort=False)

Text vocab size = 18260
Label vocab size = 3


In [0]:
def sample(probs, temp):
    probs = F.log_softmax(probs.squeeze(), dim=0)
    probs = (probs / temp).exp()
    probs /= probs.sum()
    probs = probs.cpu().numpy()

    return np.random.choice(np.arange(len(probs)), p=probs)


def generate(model, label, temp=0.6):
    model.eval()
    with torch.no_grad():        
        prev_token = train_iter.dataset.fields['text'].vocab.stoi['<s>']
        end_token = train_iter.dataset.fields['text'].vocab.stoi['</s>']
        
        hidden = None
        for _ in range(150):
            probs, hidden = model(LongTensor([[prev_token]]), LongTensor([label]),hidden)
            prev_token = sample(probs, temp)
            
            if prev_token == end_token:
                return
            
            if train_iter.dataset.fields['text'].vocab.itos[prev_token] == '\\n':
                print()
            else:
                print(train_iter.dataset.fields['text'].vocab.itos[prev_token], end=' ')

In [0]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(model, criterion, data_iter, unk_idx, pad_idx, optimizer=None, name=None):
    epoch_loss = 0
    
    is_train = not optimizer is None
    name = name or ''
    model.train(is_train)
    
    batches_count = len(data_iter)
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):                
                logits, _ = model(batch.text, batch.label)

                targets = torch.cat((batch.text[1:], batch.text.new_ones((1, batch.text.shape[1])))).view(-1)

                loss = criterion(logits.view(-1, logits.shape[-1]), targets)
                
                mask = (1 - ((targets == unk_idx) + (targets == pad_idx))).float()
                
                loss = (loss * mask).sum() / mask.sum()
                
                epoch_loss += loss.item()

                if optimizer:
                    optimizer.zero_grad()
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), 1.)
                    optimizer.step()

                progress_bar.update()
                progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(name, loss.item(), 
                                                                                         math.exp(loss.item())))
                
            progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(
                name, epoch_loss / batches_count, math.exp(epoch_loss / batches_count))
            )
            progress_bar.refresh()

    return epoch_loss / batches_count


def fit(model, criterion, optimizer, train_iter, epochs_count=1, unk_idx=0, pad_idx=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        train_loss = do_epoch(model, criterion, train_iter, unk_idx, pad_idx, optimizer, name_prefix + 'Train:')
        
        if not val_iter is None:
            val_loss = do_epoch(model, criterion, val_iter, unk_idx, pad_idx, None, name_prefix + '  Val:')
            
            if best_val_loss and val_loss > best_val_loss:
                optimizer.param_groups[0]['lr'] /= 4.
                print('Optimizer lr = {:g}'.format(optimizer.param_groups[0]['lr']))
            else:
                best_val_loss = val_loss

In [0]:
class LargeModel(nn.Module):
    def __init__(self, vocab_size, emb_dim=512, lstm_hidden_dim=512, num_layers=1):
        super().__init__()

        self._dropout = nn.Dropout(0.4)
        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.LSTM(input_size=emb_dim, hidden_size=lstm_hidden_dim, num_layers=num_layers)
        
        self._out_layer = nn.Linear(lstm_hidden_dim+1, vocab_size)
        
        self._init_weights()

    def _init_weights(self, init_range=0.1):
        self._emb.weight.data.uniform_(-init_range, init_range)
        self._out_layer.bias.data.zero_()
        self._out_layer.weight.data.uniform_(-init_range, init_range)

    def forward(self, inputs, labels, hidden=None):
        embs = self._emb(inputs)
        output, hidden = self._rnn(embs, hidden)
        labels = labels.unsqueeze(0).unsqueeze(2).expand(output.shape[0], output.shape[1], 1).float()
        output = torch.cat((output, labels), -1)
        return self._out_layer(self._dropout(output)), hidden

In [20]:
model = LargeModel(vocab_size=len(train_iter.dataset.fields['text'].vocab)).to(DEVICE)

pad_idx = train_iter.dataset.fields['text'].vocab.stoi['<pad>']
unk_idx = train_iter.dataset.fields['text'].vocab.stoi['<unk>']
criterion = nn.CrossEntropyLoss(reduction='none').to(DEVICE)

optimizer = optim.SGD(model.parameters(), lr=20., weight_decay=1e-6)

fit(model, criterion, optimizer, train_iter, epochs_count=30, unk_idx=unk_idx, pad_idx=pad_idx, val_iter=test_iter)

[5 / 30] Train: Loss = 4.35676, PPX = 78.00:   0%|          | 1/2063 [00:00<05:48,  5.92it/s]

Optimizer lr = 5


[10 / 30] Train: Loss = 3.89562, PPX = 49.19:   0%|          | 1/2063 [00:00<05:56,  5.79it/s]

Optimizer lr = 1.25


[13 / 30] Train: Loss = 3.80151, PPX = 44.77:   0%|          | 1/2063 [00:00<06:07,  5.61it/s]

...

Optimizer lr = 4.65661e-09


[30 / 30] Train: Loss = 3.75155, PPX = 42.59:   0%|          | 1/2063 [00:00<05:38,  6.08it/s]

Optimizer lr = 1.16415e-09


[30 / 30]   Val: Loss = 4.38587, PPX = 80.31: 100%|██████████| 58/58 [00:03<00:00, 15.73it/s]

Optimizer lr = 2.91038e-10


In [22]:
generate(model, 0)

мы шли в атаку на трамвае 
и я хочу тебя как в той 
и что в меня я не заметил 
что в этот день тебя люблю 


In [24]:
generate(model, 1)

я ржу ищу тебя не буду 
и не могу уснуть на грудь 
и не прощу с собой а в уши 
к концу 


### Variational & word dropout

**Задание** На прошлом занятии приводились примеры более приспособленных к RNN'ам dropout'ов. Добавьте их.

In [0]:
from scipy.stats import bernoulli

class LockedDropout(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, inputs, dropout=0.5):
        if not self.training or not dropout:
            return inputs
        
        mask = FloatTensor(bernoulli.rvs(1 - dropout, size=(1, inputs.shape[1], inputs.shape[2])) / (1 - dropout))
        return mask * inputs

In [0]:
class LargeModel(nn.Module):
    def __init__(self, vocab_size, emb_dim=512, lstm_hidden_dim=512, num_layers=1):
        super().__init__()

        self._dropout = LockedDropout()
        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.LSTM(input_size=emb_dim, hidden_size=lstm_hidden_dim, num_layers=num_layers)
        
        self._out_layer = nn.Linear(lstm_hidden_dim+1, vocab_size)
        
        self._init_weights()

    def _init_weights(self, init_range=0.1):
        self._emb.weight.data.uniform_(-init_range, init_range)
        self._out_layer.bias.data.zero_()
        self._out_layer.weight.data.uniform_(-init_range, init_range)

    def forward(self, inputs, labels, hidden=None):
        embs = self._emb(inputs)
        output, hidden = self._rnn(embs, hidden)
        labels = labels.unsqueeze(0).unsqueeze(2).expand(output.shape[0], output.shape[1], 1).float()
        output = torch.cat((output, labels), -1)
        return self._out_layer(self._dropout(output)), hidden

In [40]:
model = LargeModel(vocab_size=len(train_iter.dataset.fields['text'].vocab)).to(DEVICE)

pad_idx = train_iter.dataset.fields['text'].vocab.stoi['<pad>']
unk_idx = train_iter.dataset.fields['text'].vocab.stoi['<unk>']
criterion = nn.CrossEntropyLoss(reduction='none').to(DEVICE)

optimizer = optim.SGD(model.parameters(), lr=20., weight_decay=1e-6)

fit(model, criterion, optimizer, train_iter, epochs_count=30, unk_idx=unk_idx, pad_idx=pad_idx, val_iter=test_iter)

[12 / 30] Train: Loss = 3.97066, PPX = 53.02:   0%|          | 1/2063 [00:00<05:54,  5.81it/s]

Optimizer lr = 5


[14 / 30] Train: Loss = 3.69972, PPX = 40.44:   0%|          | 1/2063 [00:00<06:04,  5.66it/s]

Optimizer lr = 1.25


[17 / 30] Train: Loss = 3.42182, PPX = 30.63:   0%|          | 1/2063 [00:00<05:45,  5.96it/s]

...

Optimizer lr = 1.86265e-08


[30 / 30] Train: Loss = 3.56372, PPX = 35.29:   0%|          | 1/2063 [00:00<06:01,  5.71it/s]

Optimizer lr = 4.65661e-09


[30 / 30]   Val: Loss = 4.36682, PPX = 78.79: 100%|██████████| 58/58 [00:03<00:00, 15.55it/s]


Optimizer lr = 1.16415e-09


In [41]:
generate(model, 0)

олег лежит на дне колодца 
а в нём картинку и с ней снег 
еще один живет мужчина 
и с ним на улице исус 


In [49]:
generate(model, 1)

я на тебя смотрю на небо 
и вижу что ты мне за то 
что я люблю тебя родная 
как ты 


**Задание** Кроме этого, попробуйте увеличивать размер модели или количество слоев в ней, чтобы улучшить качество.

In [0]:
class LargeModel(nn.Module):
    def __init__(self, vocab_size, emb_dim=1024, lstm_hidden_dim=1024, num_layers=1):
        super().__init__()

        self._dropout = LockedDropout()
        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.LSTM(input_size=emb_dim, hidden_size=lstm_hidden_dim, num_layers=num_layers)
        self._low_out_layer = nn.Linear(lstm_hidden_dim + 1, 1024)
        self._high_out_layer = nn.Linear(1024, vocab_size)
        self._init_weights()

    def _init_weights(self, init_range=0.1):
        self._emb.weight.data.uniform_(-init_range, init_range)

    def forward(self, inputs, labels, hidden=None):
        embs = self._emb(inputs)
        output, hidden = self._rnn(embs, hidden)
        labels = labels.unsqueeze(0).unsqueeze(2).expand(output.shape[0], output.shape[1], 1).float()
        output = torch.cat((output, labels), -1)
        output = self._low_out_layer(self._dropout(output))
        return self._high_out_layer(self._dropout(output)), hidden

In [52]:
model = LargeModel(vocab_size=len(train_iter.dataset.fields['text'].vocab)).to(DEVICE)

pad_idx = train_iter.dataset.fields['text'].vocab.stoi['<pad>']
unk_idx = train_iter.dataset.fields['text'].vocab.stoi['<unk>']
criterion = nn.CrossEntropyLoss(reduction='none').to(DEVICE)

optimizer = optim.Adam(model.parameters())

fit(model, criterion, optimizer, train_iter, epochs_count=15, unk_idx=unk_idx, pad_idx=pad_idx, val_iter=test_iter)

  0%|          | 0/2063 [00:00<?, ?it/s]

Optimizer lr = 0.00025


  0%|          | 0/2063 [00:00<?, ?it/s]

Optimizer lr = 6.25e-05


  0%|          | 0/2063 [00:00<?, ?it/s]

...

Optimizer lr = 1.52588e-08


  0%|          | 0/2063 [00:00<?, ?it/s]

Optimizer lr = 3.8147e-09


[15 / 15]   Val: Loss = 4.64122, PPX = 103.67: 100%|██████████| 58/58 [00:06<00:00,  8.24it/s]

Optimizer lr = 9.53674e-10


In [53]:
generate(model, 0)

я не люблю тебя родная 
я буду больше никогда 
как я не знаю как обычно 
а ты и вправду человек 


In [54]:
generate(model, 1)

олег на голову собрался 
в слезах олег кричит ура 
и спрашивает папа с криком 
давай 


## Multi-task learning

Ещё один важный способ улучшения модели - multi-task learning. Это когда одна модель учится делать предсказания сразу для нескольких задач.

В нашем случае это может быть предсказанием отдельно леммы слова и отдельно - его грамматического значения:
![](https://hsto.org/web/e97/8a8/6e8/e978a86e8a874d8d946bb15e6a49a713.png =x350)

В итоге модель выучивает как языковую модель по леммам, так и модель POS tagging'а. Одновременно!

Возьмем корпус из universal dependencies - он уже размечен, как нужно.

Почитаем его:

In [0]:
from corpus_iterator import Token, CorpusIterator

fields = [('word', Field()), ('lemma', Field()), ('gram_val', Field())]
examples = []

with CorpusIterator('UD_Russian-SynTagRus/ru_syntagrus-ud-train.conllu') as corpus_iter:
    for sent in corpus_iter:
        words = ['<s>'] + [tok.token.lower() for tok in sent] + ['</s>']
        lemmas = ['<s>'] + [tok.lemma.lower() for tok in sent] + ['</s>']
        gr_vals = ['<s>'] + [tok.grammar_value for tok in sent] + ['</s>']
        examples.append(Example.fromlist([words, lemmas, gr_vals], fields))

In [56]:
print('Words:', examples[1].word)
print('Lemmas:', examples[1].lemma)
print('Grammar vals:', examples[1].gram_val)

Words: ['<s>', 'начальник', 'областного', 'управления', 'связи', 'семен', 'еремеевич', 'был', 'человек', 'простой', ',', 'приходил', 'на', 'работу', 'всегда', 'вовремя', ',', 'здоровался', 'с', 'секретаршей', 'за', 'руку', 'и', 'иногда', 'даже', 'писал', 'в', 'стенгазету', 'заметки', 'под', 'псевдонимом', '"', 'муха', '"', '.', '</s>']
Lemmas: ['<s>', 'начальник', 'областной', 'управление', 'связь', 'семен', 'еремеевич', 'быть', 'человек', 'простой', ',', 'приходить', 'на', 'работа', 'всегда', 'вовремя', ',', 'здороваться', 'с', 'секретарша', 'за', 'рука', 'и', 'иногда', 'даже', 'писать', 'в', 'стенгазета', 'заметка', 'под', 'псевдоним', '"', 'муха', '"', '.', '</s>']
Grammar vals: ['<s>', 'NOUN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing', 'ADJ|Case=Gen|Degree=Pos|Gender=Neut|Number=Sing', 'NOUN|Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing', 'NOUN|Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing', 'PROPN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing', 'PROPN|Animacy=Anim|Case=Nom|Gende

Таким образом, размер словаря может быть существенно сокращен - лемм меньше, чем слов, а предсказание грамматики вынуждает модель быть более осведомленной о согласовании слов.

In [57]:
dataset = Dataset(examples, fields)

dataset.fields['word'].build_vocab(dataset, min_freq=3)
print('Word vocab size =', len(dataset.fields['word'].vocab))
dataset.fields['lemma'].build_vocab(dataset, min_freq=3)
print('Lemma vocab size =', len(dataset.fields['lemma'].vocab))
dataset.fields['gram_val'].build_vocab(dataset)
print('Grammar val vocab size =', len(dataset.fields['gram_val'].vocab))

train_dataset, test_dataset = dataset.split(split_ratio=0.75)

train_iter, test_iter = BucketIterator.splits(datasets=(train_dataset, test_dataset), batch_sizes=(32, 128), 
                                              shuffle=True, device=DEVICE, sort=False)

Word vocab size = 29984
Lemma vocab size = 17059
Grammar val vocab size = 737


Построим маппинг из пары (лемма, грамматическое значение) в слово - если бы у нас под рукой был морфологический словарь, маппинг можно было бы пополнить, добавить слова для лемм из корпуса, которые не встретились в обучении.

In [0]:
dictionary = {
    (lemma, gram_val): word
    for example in train_iter.dataset.examples 
    for word, lemma, gram_val in zip(example.word, example.lemma, example.gram_val)
}

**Задание**  Обновите генератор - например, можно сэмплировать лемму и находить самое вероятное грамматическое значение, которое встречается  в паре с этой леммой в `dictionary`.

In [0]:
def generate(model, temp=0.7):
    model.eval()
    with torch.no_grad():
        prev_lemma = train_iter.dataset.fields['lemma'].vocab.stoi['<s>']
        end_lemma = train_iter.dataset.fields['lemma'].vocab.stoi['</s>']
        prev_gram_val = train_iter.dataset.fields['gram_val'].vocab.stoi['<s>']
        end_gram_val = train_iter.dataset.fields['gram_val'].vocab.stoi['</s>']
        
        hidden = None
        for _ in range(150):
            lemma_probs, gram_val_probs, hidden = model(LongTensor([[prev_lemma]]), LongTensor([[prev_gram_val]]), hidden)
            prev_lemma = sample(lemma_probs, temp)
            
            word = None
            
            for i in np.argsort(gram_val_probs).cpu().numpy()[0][0][::-1]:
                try:
                    word = dictionary[train_iter.dataset.fields['lemma'].vocab.itos[prev_lemma],
                                      train_iter.dataset.fields['gram_val'].vocab.itos[i]]
                    prev_gram_val = i
                except KeyError:
                    pass
            
            if (prev_lemma == end_lemma) or (prev_gram_val == end_gram_val):
                return
            
            print(word, end=' ')

**Задание** Обновите модель и функцию обучения.

Модель должна принимать пары `lemma, gr_val`, конкатенировать их эмбеддинги и предсказывать следующие `lemma, gr_val` по выходу из LSTM.

Функция `do_epoch` должна суммировать потери по предсказанию леммы (делая маскинг для `<unk>` и `<pad>`) + потери по предсказанию грамматического значения (с маскингом по `<pad>`).

In [0]:
class MultitaskLearningLMModel(nn.Module):
    def __init__(self, vocab_size, gr_val_size, emb_dim=512, lstm_hidden_dim=512, num_layers=1):
        super().__init__()

        self._lemma_emb = nn.Embedding(vocab_size, emb_dim)
        self._gr_val_emb = nn.Embedding(gr_val_size, emb_dim)
        self._rnn = nn.LSTM(input_size=2*emb_dim, hidden_size=lstm_hidden_dim, num_layers=num_layers)
        
        self._lemma_out_layer = nn.Linear(lstm_hidden_dim, vocab_size)
        self._gr_val_out_layer = nn.Linear(lstm_hidden_dim, gr_val_size)
        
        self._init_weights()

    def _init_weights(self, init_range=0.1):
        self._lemma_emb.weight.data.uniform_(-init_range, init_range)
        self._lemma_out_layer.bias.data.zero_()
        self._lemma_out_layer.weight.data.uniform_(-init_range, init_range)
        
        self._gr_val_emb.weight.data.uniform_(-init_range, init_range)
        self._gr_val_out_layer.bias.data.zero_()
        self._gr_val_out_layer.weight.data.uniform_(-init_range, init_range)

    def forward(self, lemmas, gram_vals, hidden=None):
        lemma_embs = self._lemma_emb(lemmas)
        gr_val_embs = self._gr_val_emb(gram_vals)
        
        embs = torch.cat((lemma_embs, gr_val_embs), -1)
        
        output, hidden = self._rnn(embs, hidden)
        lemma_out = self._lemma_out_layer(output)
        gr_val_out = self._gr_val_out_layer(output)
        return lemma_out, gr_val_out, hidden

In [0]:
batch = next(iter(train_iter))

In [62]:
model = MultitaskLearningLMModel(vocab_size=len(train_iter.dataset.fields['lemma'].vocab),
                                 gr_val_size=len(train_iter.dataset.fields['gram_val'].vocab)).to(DEVICE)

model(batch.lemma, batch.gram_val)

(tensor([[[ 0.0041, -0.0030,  0.0415,  ..., -0.0023, -0.0255, -0.0090],
          [ 0.0041, -0.0030,  0.0415,  ..., -0.0023, -0.0255, -0.0090],
          [ 0.0041, -0.0030,  0.0415,  ..., -0.0023, -0.0255, -0.0090],
          ...,
          [ 0.0041, -0.0030,  0.0415,  ..., -0.0023, -0.0255, -0.0090],
          [ 0.0041, -0.0030,  0.0415,  ..., -0.0023, -0.0255, -0.0090],
          [ 0.0041, -0.0030,  0.0415,  ..., -0.0023, -0.0255, -0.0090]],
 
         [[-0.0125, -0.0014,  0.0141,  ..., -0.0143, -0.0084, -0.0096],
          [ 0.0121, -0.0043,  0.0406,  ..., -0.0074, -0.0156, -0.0305],
          [-0.0060,  0.0048,  0.0391,  ...,  0.0092, -0.0245, -0.0268],
          ...,
          [-0.0097, -0.0097,  0.0383,  ...,  0.0098, -0.0330, -0.0398],
          [-0.0097, -0.0097,  0.0383,  ...,  0.0098, -0.0330, -0.0398],
          [ 0.0004, -0.0132,  0.0187,  ..., -0.0123, -0.0392,  0.0120]],
 
         [[-0.0289,  0.0102,  0.0286,  ..., -0.0093, -0.0051,  0.0123],
          [ 0.0038,  0.0009,

In [0]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(model, criterion, data_iter, unk_idx, pad_idx, optimizer=None, name=None):
    epoch_loss = 0
    
    is_train = not optimizer is None
    name = name or ''
    model.train(is_train)
    
    batches_count = len(data_iter)
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):                
                lemma_logits, gram_val_logits, _ = model(batch.lemma, batch.gram_val)

                lemma_targets = torch.cat((batch.lemma[1:], batch.lemma.new_ones((1, batch.lemma.shape[1])))).view(-1)
                gram_val_targets = torch.cat((batch.gram_val[1:], batch.gram_val.new_ones((1, batch.gram_val.shape[1])))).view(-1)

                lemma_loss = criterion(lemma_logits.view(-1, lemma_logits.shape[-1]), lemma_targets)
                gram_val_loss = criterion(gram_val_logits.view(-1, gram_val_logits.shape[-1]), gram_val_targets)
                
                lemma_mask = (1 - ((lemma_targets == unk_idx) + (lemma_targets == pad_idx))).float()
                gram_val_mask = (1 - (gram_val_targets == pad_idx)).float()
                
                loss = ((lemma_loss * lemma_mask).sum() / lemma_mask.sum()) + ((gram_val_loss * gram_val_mask).sum() / gram_val_mask.sum())
                
                epoch_loss += loss.item()

                if optimizer:
                    optimizer.zero_grad()
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), 1.)
                    optimizer.step()

                progress_bar.update()
                progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(name, loss.item(), 
                                                                                         math.exp(loss.item())))
                
            progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(
                name, epoch_loss / batches_count, math.exp(epoch_loss / batches_count))
            )
            progress_bar.refresh()

    return epoch_loss / batches_count


def fit(model, criterion, optimizer, train_iter, epochs_count=1, unk_idx=0, pad_idx=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        train_loss = do_epoch(model, criterion, train_iter, unk_idx, pad_idx, optimizer, name_prefix + 'Train:')
        
        if not val_iter is None:
            val_loss = do_epoch(model, criterion, val_iter, unk_idx, pad_idx, None, name_prefix + '  Val:')
            
            if best_val_loss and val_loss > best_val_loss:
                optimizer.param_groups[0]['lr'] /= 4.
                print('Optimizer lr = {:g}'.format(optimizer.param_groups[0]['lr']))
            else:
                best_val_loss = val_loss
        print()
        generate(model)
        print()

In [65]:
# I train only 10 epoches, because the model starts overfitting
model = MultitaskLearningLMModel(vocab_size=len(train_iter.dataset.fields['lemma'].vocab),
                                 gr_val_size=len(train_iter.dataset.fields['gram_val'].vocab)).to(DEVICE)

pad_idx = train_iter.dataset.fields['lemma'].vocab.stoi['<pad>']
unk_idx = train_iter.dataset.fields['lemma'].vocab.stoi['<unk>']
criterion = nn.CrossEntropyLoss(reduction='none').to(DEVICE)

optimizer = optim.SGD(model.parameters(), lr=20., weight_decay=1e-6)

fit(model, criterion, optimizer, train_iter, epochs_count=10, unk_idx=unk_idx, pad_idx=pad_idx, val_iter=test_iter)

  0%|          | 0/1145 [00:00<?, ?it/s]


этом получался , что этом этом _ ! " 


[3 / 10] Train: Loss = 7.68100, PPX = 2166.79:   0%|          | 1/1145 [00:00<03:06,  6.13it/s]


не бывший . 


  0%|          | 0/1145 [00:00<?, ?it/s]

...

Optimizer lr = 1.25

практическом этим нас ходившей не понимался , что в условиями зарплате энергии никакой не ставших . 


  0%|          | 0/1145 [00:00<?, ?it/s]

Optimizer lr = 0.3125

в рядами случаем , после называемым немецкой фирмы , найдешь и. том , что нас всех политическом , скорее всех , в такой не по , котором еще будьте делах . 


[10 / 10]   Val: Loss = 8.34652, PPX = 4215.49: 100%|██████████| 96/96 [00:13<00:00,  7.50it/s]


Optimizer lr = 0.078125

как сообщило " риа - новостями " в - предсказания , " не бывшую на базам " , а , химии и. названных нацию " . 


## Контролируемая генерация

Хочется сделать генерацию более контролируемой - в идеале, задавать тему.

Простой способ - сделать тематическое моделирование и найти в текстах какие-то темы - а потом передавать вектор тем вместе с эмбеддингом слова, чтобы модель училась генерировать тематически-согласованный текст.

In [0]:
def read_poem(path):
    poem = []
    with open(path, encoding='utf8') as f:
        for line in f:
            line = line.rstrip()
            if len(line) == 0:
                yield poem
                poem = []
                continue
            
            poem.extend(line.split() + ['\\n'])
            
perashki = list(read_poem('perashki.txt'))

In [0]:
from gensim import corpora, models

docs = [[word for word in poem if word != '\\n'] for poem in perashki]

dictionary = corpora.Dictionary(docs)
dictionary.filter_n_most_frequent(100)

bow_corpus = [dictionary.doc2bow(doc) for doc in docs]

lda_model = models.LdaModel(bow_corpus, num_topics=5, id2word=dictionary, passes=5)

Посмотреть, что выучилось, можно так:

In [68]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      22.560722        1       1  0.084680  0.107589
4      20.748171        1       2  0.129541  0.122547
2      20.242851        1       3  0.032515 -0.213671
0      19.175110        1       4  0.003210 -0.087508
1      17.273140        1       5 -0.249947  0.071043, topic_info=      Category        Freq     Term       Total  loglift  logprob
term                                                             
687    Default  544.000000      лет  544.000000  30.0000  30.0000
488    Default  295.000000     снег  295.000000  29.0000  29.0000
442    Default  523.000000   любовь  523.000000  28.0000  28.0000
4213   Default  475.000000      год  475.000000  27.0000  27.0000
49     Default  305.000000    купил  305.000000  26.0000  26.0000
5356   Default  193.000000     кофе  193.000000  25.0000  25.0000
187    Default  373.000000    всего  373.000000  24.0000  24.0000
358    Default  216.000000    сорок  216.000000  23.0000  23.0000
25     Default  311.000000     окно  311.000000  22.0000  22.0000
513    Default  382.000000   четыре  382.000000  21.0000  21.0000
1089   Default  171.000000    затем  171.000000  20.0000  20.0000
1963   Default  222.000000    пусть  222.000000  19.0000  19.0000
4439   Default  573.000000     чуть  573.000000  18.0000  18.0000
3840   Default  244.000000   бывает  244.000000  17.0000  17.0000
1039   Default  348.000000     буду  348.000000  16.0000  16.0000
848    Default  262.000000    пишет  262.000000  15.0000  15.0000
11289  Default  169.000000    осень  169.000000  14.0000  14.0000
752    Default  453.000000    жизни  453.000000  13.0000  13.0000
147    Default  420.000000    никто  420.000000  12.0000  12.0000
1296   Default  318.000000     друг  318.000000  11.0000  11.0000
82     Default  512.000000  человек  512.000000  10.0000  10.0000
260    Default  384.000000  смотрит  384.000000   9.0000   9.0000
588    Default  256.000000     этом  256.000000   8.0000   8.0000
791    Default  251.000000   вокруг  251.000000   7.0000   7.0000
3451   Default  204.000000     года  204.000000   6.0000   6.0000
2606   Default  153.000000   доктор  153.000000   5.0000   5.0000
777    Default  314.000000     хоть  314.000000   4.0000   4.0000
1000   Default  332.000000    детей  332.000000   3.0000   3.0000
932    Default  136.000000   пришол  136.000000   2.0000   2.0000
4005   Default  277.000000   солнце  277.000000   1.0000   1.0000
...        ...         ...      ...         ...      ...      ...
1061    Topic5   67.652985  детства   68.352272   1.7457  -7.2418
555     Topic5   66.458794    пошли   67.155029   1.7456  -7.2596
6734    Topic5   66.419060    зимой   67.125595   1.7454  -7.2602
8858    Topic5   65.307137   покажу   66.016243   1.7452  -7.2771
6008    Topic5   65.490761     дому   66.204788   1.7452  -7.2743
5068    Topic5   63.417198       ль   64.113037   1.7451  -7.3065
2662    Topic5   64.587769    снегу   65.298607   1.7451  -7.2882
49      Topic5  209.946136    купил  305.894592   1.3796  -6.1093
2759    Topic5   94.764824   случай  107.570328   1.6293  -6.9048
7503    Topic5   96.211990   других  119.661331   1.5379  -6.8896
6739    Topic5  140.108017    глеба  244.626129   1.1987  -6.5138
2865    Topic5  107.390327     воды  166.953674   1.3148  -6.7797
1000    Topic5  142.089081    детей  332.746185   0.9051  -6.4997
1803    Topic5  129.275467    можно  406.761414   0.6097  -6.5942
1388    Topic5  141.171036    опять  529.913147   0.4333  -6.5062
957     Topic5  131.940674    давай  444.928467   0.5405  -6.5738
3755    Topic5   95.892929   женщин  172.919800   1.1664  -6.8930
366     Topic5  126.881783    люблю  458.589508   0.4711  -6.6129
493     Topic5  121.026543     куда  401.724640   0.5563  -6.6602
151     Topic5  121.890923    будто  461.827484   0.4240  -6.6531
25      Topic5  107.009956     окно  311.075623   0.6889  -6.7833
131     Topic5 

Предсказывает распределение модель как-то так:

In [69]:
for word in perashki[10]:
    if word == '\\n':
        print()
    else:
        print(word, end=' ')

олег ваквариуме шарит 
глазами полными беды 
а гдеж моя уитни хьюстон 
у грота нет в кормушке нет 


In [70]:
lda_model.get_document_topics(bow_corpus[10])

[(0, 0.7048466),
 (1, 0.016812785),
 (2, 0.24477454),
 (3, 0.016765906),
 (4, 0.016800195)]

**Задание** Посчитайте для всех текстов вектора тем, передавайте их вместе со словами (конкатенируя к эмбеддингам). Посмотрите, вдруг чего получится.

In [0]:
themes = []
for text in bow_corpus:
    topics = lda_model.get_document_topics(text)
    
    # According to gensim docs it is possible that less than num_topics
    # themes are extracted. That's why I decided to fill blanks with
    # uniform-distributed residue
    if len(topics) != 5:
        total = 0
        for topic in topics:
            total += topic[1]
        additional = (1 - total) / (5 - len(topics))
        candidates = set(range(5))
        for topic in topics:
            candidates.remove(topic[0])
        for candidate in candidates:
            topics.append((candidate, additional))
        
    themes.append(np.array(topics)[:, 1])

In [0]:
from torchtext.data import Field, Example, Dataset, BucketIterator

text_field = Field(init_token='<s>', eos_token='</s>')
theme_field = Field(sequential=False, use_vocab=False, dtype=torch.float64)
        
fields = [('text', text_field), ('theme', theme_field)]
examples = [Example.fromlist([poem, theme], fields) for poem, theme in zip(perashki, themes)]
dataset = Dataset(examples, fields)

text_field.build_vocab(dataset, min_freq=7)

train_dataset, test_dataset = dataset.split(split_ratio=0.9)

train_iter, test_iter = BucketIterator.splits(datasets=(train_dataset, test_dataset), batch_sizes=(32, 128), 
                                              shuffle=True, device=DEVICE, sort=False)

In [0]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(model, criterion, data_iter, unk_idx, pad_idx, optimizer=None, name=None):
    epoch_loss = 0
    
    is_train = not optimizer is None
    name = name or ''
    model.train(is_train)
    
    batches_count = len(data_iter)
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):                
                logits, _ = model(batch.text, batch.theme)

                targets = torch.cat((batch.text[1:], batch.text.new_ones((1, batch.text.shape[1])))).view(-1)

                loss = criterion(logits.view(-1, logits.shape[-1]), targets)
                
                mask = (1 - ((targets == unk_idx) + (targets == pad_idx))).float()
                
                loss = (loss * mask).sum() / mask.sum()
                
                epoch_loss += loss.item()

                if optimizer:
                    optimizer.zero_grad()
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), 1.)
                    optimizer.step()

                progress_bar.update()
                progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(name, loss.item(), 
                                                                                         math.exp(loss.item())))
                
            progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(
                name, epoch_loss / batches_count, math.exp(epoch_loss / batches_count))
            )
            progress_bar.refresh()

    return epoch_loss / batches_count


def fit(model, criterion, optimizer, train_iter, epochs_count=1, unk_idx=0, pad_idx=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        train_loss = do_epoch(model, criterion, train_iter, unk_idx, pad_idx, optimizer, name_prefix + 'Train:')
        
        if not val_iter is None:
            val_loss = do_epoch(model, criterion, val_iter, unk_idx, pad_idx, None, name_prefix + '  Val:')
            
            if best_val_loss and val_loss > best_val_loss:
                optimizer.param_groups[0]['lr'] /= 4.
                print('Optimizer lr = {:g}'.format(optimizer.param_groups[0]['lr']))
            else:
                best_val_loss = val_loss

In [0]:
class LargeModel(nn.Module):
    def __init__(self, vocab_size, emb_dim=512, lstm_hidden_dim=512, num_layers=1):
        super().__init__()

        self._dropout = nn.Dropout(0.4)
        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.LSTM(input_size=emb_dim+5, hidden_size=lstm_hidden_dim, num_layers=num_layers)
        
        self._out_layer = nn.Linear(lstm_hidden_dim, vocab_size)
        
        self._init_weights()

    def _init_weights(self, init_range=0.1):
        self._emb.weight.data.uniform_(-init_range, init_range)
        self._out_layer.bias.data.zero_()
        self._out_layer.weight.data.uniform_(-init_range, init_range)

    def forward(self, inputs, theme, hidden=None):
        embs = self._emb(inputs)
        theme = theme.unsqueeze(0).expand(embs.shape[0], embs.shape[1], theme.shape[1]).float()
        embs = torch.cat((embs, theme), -1)
        output, hidden = self._rnn(embs, hidden)
        return self._out_layer(self._dropout(output)), hidden

In [75]:
model = LargeModel(vocab_size=len(train_iter.dataset.fields['text'].vocab)).to(DEVICE)

pad_idx = train_iter.dataset.fields['text'].vocab.stoi['<pad>']
unk_idx = train_iter.dataset.fields['text'].vocab.stoi['<unk>']
criterion = nn.CrossEntropyLoss(reduction='none').to(DEVICE)

optimizer = optim.Adam(model.parameters())

fit(model, criterion, optimizer, train_iter, epochs_count=30, unk_idx=unk_idx, pad_idx=pad_idx, val_iter=test_iter)

[9 / 30] Train: Loss = 2.91736, PPX = 18.49:   0%|          | 2/1386 [00:00<02:46,  8.32it/s]

Optimizer lr = 0.00025


[11 / 30] Train: Loss = 2.68224, PPX = 14.62:   0%|          | 2/1386 [00:00<02:52,  8.02it/s]

Optimizer lr = 6.25e-05


[12 / 30] Train: Loss = 2.57315, PPX = 13.11:   0%|          | 2/1386 [00:00<02:54,  7.95it/s]

...

Optimizer lr = 9.09495e-16


[30 / 30] Train: Loss = 2.63248, PPX = 13.91:   0%|          | 2/1386 [00:00<02:47,  8.27it/s]

Optimizer lr = 2.27374e-16


[30 / 30]   Val: Loss = 4.19735, PPX = 66.51: 100%|██████████| 39/39 [00:01<00:00, 19.73it/s]

Optimizer lr = 5.68434e-17


In [0]:
def generate(model, theme, temp=0.6):
    model.eval()
    with torch.no_grad():        
        prev_token = train_iter.dataset.fields['text'].vocab.stoi['<s>']
        end_token = train_iter.dataset.fields['text'].vocab.stoi['</s>']
        
        hidden = None
        for _ in range(150):
            probs, hidden = model(LongTensor([[prev_token]]), LongTensor([theme]), hidden)
            prev_token = sample(probs, temp)
            
            if prev_token == end_token:
                return
            
            if train_iter.dataset.fields['text'].vocab.itos[prev_token] == '\\n':
                print()
            else:
                print(train_iter.dataset.fields['text'].vocab.itos[prev_token], end=' ')

In [77]:
generate(model, [1, 0, 0, 0, 0])

я выпил всех на каждом доме 
и год идет один в тюрьме 
и там в москве на шпильках в 
в пустом вагоне за стеклом 


In [78]:
generate(model, [0.3, 0, 0, 0.7, 0])

оксана говорит что это 
а не поможет никогда 
и только я к тебе бы больше 
послать не приходи ко мне 


# Дополнительные материалы

## Статьи

Regularizing and Optimizing LSTM Language Models, 2017 [[arxiv]](https://arxiv.org/abs/1708.02182), [[github]](https://github.com/salesforce/awd-lstm-lm) - одна из самых полезных статей про языковые модели + репозиторий, в котором реализовано много полезного, стоит заглянуть

Exploring the Limits of Language Modeling, 2016 [[arxiv]](https://arxiv.org/abs/1602.02410)

Using the Output Embedding to Improve Language Models, 2017 [[pdf]](http://www.aclweb.org/anthology/E17-2025)


## Transfer learning
[Transfer learning, cs231n](http://cs231n.github.io/transfer-learning/)  
[Transfer learning, Ruder](http://ruder.io/transfer-learning/) - очень подробная статья от чувака из NLP

## Multi-task learning
[An Overview of Multi-Task Learning in Deep Neural Networks, Ruder](http://ruder.io/multi-task/)  
[Multi-Task Learning Objectives for Natural Language Processing, Ruder](http://ruder.io/multi-task-learning-nlp/)

# Сдача

[Форма для сдачи](https://goo.gl/forms/ASLLjYncKUcIHmuO2)  
[Feedback](https://goo.gl/forms/9aizSzOUrx7EvGlG3)